In [1]:
import requests
import json
from IPython.display import JSON
from datetime import datetime
import pandas as pd
from difflib import get_close_matches

In [2]:
dataframes = pd.read_csv('clean_advocacy.csv')
dataframes.head()

,county,date,Code
0,Isiolo,23/01/2023,11
1,Isiolo,31/10/2018,11
2,Marsabit,24/01/2023,10
3,Uasin Gishu,27/01/2023,27
4,Uasin Gishu,26/07/2022,27


In [3]:
county_meetings = dataframes.drop('Code', axis=1)
county_meetings['date'] = pd.to_datetime(county_meetings.date, dayfirst=True)

In [4]:
county_meetings.sample(5)

,county,date
47,Garissa,2021-06-03
51,Kisumu,2022-04-06
102,Trans Nzoia,2018-10-10
38,Nyeri,2022-02-17
62,Kilifi,2018-07-26


In [5]:
county_meetings = county_meetings.query('date < 20230501')

In [6]:
# You can find most of this code in the "Zoho API orchestration" notebook
# I will add more information in these lines for you to understand better what the code does.
import requests
import json
# get_close_matches function helps me match strings which are typed in slightly different manners
from difflib import get_close_matches

In [7]:
with open('./self_client.json') as client_info, open('./portals_info.json') as portals_info, open('./refresh_tokens.json') as tokens_file:
    client_app_data = json.load(client_info)
    zoho_portals_data = json.load(portals_info)
    tokens_dict = json.load(tokens_file)

In [8]:
payload = {
    'grant_type':'authorization_code',
    'client_id':client_app_data['client_id'],
    'client_secret':client_app_data['client_secret'],
    'code':client_app_data['code'],
    'redirect_uri':''
}

In [9]:
TOKEN_URI = 'https://accounts.zoho.com/oauth/v2/token'
PROJECTS_API_URI = f'https://projectsapi.zoho.com/restapi/portal/{zoho_portals_data["projects_portal_id"]}/projects/'
# BOOKS_API_URI = 'https://www.zohoapis.com/books/v3'
CRM_APU_URI = 'https://www.zohoapis.com/crm/v4'

In [18]:
# Often, I work in different zoho Applications at the same time.
# Each application requires different requests headers. To make my life easier, I create a requests
# session for each application and store all of them in zoho_apps_api_sessions.
# For example, when I need to work with Zoho Books api, I will take the Zoho books session.
zoho_apps_api_sessions = tokens_dict['tokens'].copy()

refresh_payload = {
    'grant_type':'refresh_token',
    'client_id':client_app_data['client_id'],
    'client_secret':client_app_data['client_secret']
}

# All sessions need to be refreshed every now and then
for scope, refresh_token in tokens_dict['tokens'].items():
    refresh_payload['refresh_token'] = refresh_token
    refresh_response = requests.post(TOKEN_URI, data=refresh_payload)
    print(scope, refresh_response)
    
    # Create app specific session
    s = requests.session()
    s.headers = {
        'Authorization':f'Bearer {refresh_response.json()["access_token"]}'
    }
    # Store in sessions dict
    zoho_apps_api_sessions[scope] = s

access_token <Response [200]>


KeyError: 'access_token'